<h1>Parsing OMA website</h1>

<p>This notebook is just a prototype for testing pieces of python code aimed to parse a specific website.</p>

In [28]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup as BSoup

In [29]:
url = 'https://www.oma.by/catalog/'
#opens up a connection, grabs the page and downloads it
oma_main_html = requests.get(url)

<p>soup is a BeautifulSoup object which represents the document as a nested data sctructure:</p>

In [30]:
oma_catalog_soup = BSoup(oma_main_html.text, 'html.parser')
oma_catalog_soup.title
oma_catalog_soup.close

<h3>Getting categories </h3>
<p>So we get the soup object successfully. Now will try to find category headers on a catalog page (there are 7 main categories). </p>
<img src="screenshots/category_schema.png" align="left" width="600">

<br>
<p>I will try to select sections with class 'border-section' select its child h2.</p>
<img src="screenshots/categories.png" align="left">

In [50]:
categories = oma_catalog_soup.select('section.bordered-section h2')
print(f'There are {len(categories)} objects that has h2 tags inside the header tag')

print('\nPrinting the whole list of the categories:')
for category in categories:
    print(category)

There are 7 objects that has h2 tags inside the header tag

Printing the whole list of the categories:
<h2>Инструменты, крепёж</h2>
<h2>Отделка</h2>
<h2>Садовый центр / отдых</h2>
<h2>Сантехника</h2>
<h2>Строительство</h2>
<h2>Товары для дома</h2>
<h2>Электротехника</h2>


<h3>Getting subcategories </h3>
<p>So we get the categories list correctly and we can use this h2 tags as anchors for looping.</p>
<img src="screenshots/subcategories_schema.png" align="left" width="720">

In [53]:
categories = oma_catalog_soup.findAll('section',\
                                      {'class':'bordered-section js-accordion-group'})
print(f'There are {len(categories)} main category objects ')

print('\nPrinting the whole list of the categories:')
print(type(categories[0]))

print(f'\nSubcategory contains {len(categories[0].text)} symbols.')


There are 7 main category objects 

Printing the whole list of the categories:
<class 'bs4.element.Tag'>

Subcategory contains 9762 symbols.


<p>Here is what one subcategory looks like:</p>
<img src="screenshots/subcategory_lvl_2.png" align="left" width="600">